In [1]:
import numpy as np
import matplotlib.pyplot as plt
import genetools
import seaborn as sns

sns.set_style("dark")

In [2]:
import pandas as pd

In [3]:
import dask
import dask.dataframe as dd

In [4]:
import os

In [5]:
from malid import config

Raise worker connection timeouts (see other dask notebooks):

In [6]:
import distributed

# These only seem to be picked up by scheduler, not by individual workers

dask.config.set(
    {
        "distributed.comm.timeouts.tcp": "120s",
        "distributed.comm.timeouts.connect": "120s",
        "distributed.comm.retry.count": 5,
    }
)

In [7]:
# These will be picked up by individual workers

with open(os.path.expandvars("$HOME/.config/dask/distributed.yaml"), "w") as w:
    w.write(
        """distributed:
  comm:
    retry:
      count: 5
    timeouts:
      connect: 120s          # time before connecting fails
      tcp: 120s              # time before calling an unresponsive connection dead
    """
    )

In [8]:
from dask.distributed import Client

# multi-processing backend
# access dashbaord at http://127.0.0.1:61083
# if already opened from another notebook, see https://stackoverflow.com/questions/60115736/dask-how-to-connect-to-running-cluster-scheduler-and-access-total-occupancy
client = Client(
    scheduler_port=61084,
    dashboard_address=":61083",
    n_workers=7,
    processes=True,
    threads_per_worker=8,
    memory_limit="auto",
    worker_dashboard_address=":0",  # start worker dashboards on random ports
)
display(client)
# for debugging: client.restart()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:61083/status,
Dashboard: http://127.0.0.1:61083/status,Workers: 7
Total threads: 56,Total memory: 1.15 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61084,Workers: 7
Dashboard: http://127.0.0.1:61083/status,Total threads: 56
Started: Just now,Total memory: 1.15 TiB
Comm: tcp://127.0.0.1:36185,Total threads: 8
Dashboard: http://127.0.0.1:45089/status,Memory: 167.92 GiB
Nanny: tcp://127.0.0.1:45929,


In [9]:
desired_cols = [
    "participant_label",
    "specimen_label",
    "disease",
    "disease_subtype",
    "specimen_time_point",
    "participant_age",
    "participant_description",
]

In [10]:
debug_filters = None
# debug_filters = [("participant_label", "==", "BFI-0007450")]

In [11]:
# Don't use fastparquet, because it changes specimen labels like M54-049 to 2049-01-01 00:00:54 -- i.e. it coerces partition names to numbers or dates
df = dd.read_parquet(
    config.paths.sequences,
    columns=desired_cols,
    filters=debug_filters,
    engine="pyarrow",
)

In [12]:
df

,participant_label,specimen_label,disease,disease_subtype,specimen_time_point,participant_age,participant_description
npartitions=717,,,,,,,
,category[known],category[known],category[unknown],category[unknown],object,object,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [13]:
# df = df.drop_duplicates()
# display(df)
# dask.visualize(df)
# this has an aggregation step

In [14]:
# use map_partitions to avoid agg step that is unnecessary based on our partitioning strategy

metadata_df = df.map_partitions(lambda part: part.drop_duplicates())

In [15]:
metadata_df

,participant_label,specimen_label,disease,disease_subtype,specimen_time_point,participant_age,participant_description
npartitions=717,,,,,,,
,category[known],category[known],category[unknown],category[unknown],object,object,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [16]:
# dask.visualize(metadata_df, filename="participant_specimen_metadata.dask_task_graph.pdf")

In [17]:
metadata_df_c = metadata_df.compute()

In [18]:
metadata_df_c

,participant_label,specimen_label,disease,disease_subtype,specimen_time_point,participant_age,participant_description
0,BFI-0000234,M124-S014,Healthy/Background,Healthy/Background - HIV Negative,None,27,Location: USA
74701,BFI-0000234,M132-S014,Healthy/Background,Healthy/Background - HIV Negative,None,27,Location: USA
0,BFI-0000254,M111-S037,HIV,HIV Broad Neutralizing,None,48,Location: Tanzania
98181,BFI-0000254,M114-S037,HIV,HIV Broad Neutralizing,None,48,Location: Tanzania
0,BFI-0000255,M111-S033,HIV,HIV Broad Neutralizing,None,33,Location: Tanzania
...,...,...,...,...,...,...,...
0,BFI-0010241,M464-S042,Healthy/Background,Healthy/Background (children),None,15,None
0,BFI-0010242,M464-S043,Healthy/Background,Healthy/Background (children),None,14,None
0,BFI-0010243,M464-S044,Healthy/Background,Healthy/Background (children),None,13,None
0,BFI-0010244,M464-S045,Healthy/Background,Healthy/Background (children),None,12,None


In [19]:
metadata_df_c = metadata_df_c.sort_values(["disease", "participant_label"])
metadata_df_c

,participant_label,specimen_label,disease,disease_subtype,specimen_time_point,participant_age,participant_description
0,BFI-0007450,M369-S001,Covid19,Covid19 - Sero-positive (ICU),9 days,None,COVID-19 project
495704,BFI-0007450,M371-S003,Covid19,Covid19 - Sero-positive (ICU),22 days,None,COVID-19 project
348716,BFI-0007450,M371-S014,Covid19,Covid19 - Sero-positive (ICU),27 days,None,COVID-19 project
460228,BFI-0007453,M369-S004,Covid19,Covid19 - Sero-negative (ICU),8 days,None,COVID-19 project
0,BFI-0007453,M369-S005,Covid19,Covid19 - Sero-negative (ICU),11 days,None,COVID-19 project
...,...,...,...,...,...,...,...
0,BFI-0010060,M456-S001,Lupus,SLE Patient,None,51,SLE Patient
0,BFI-0010061,M456-S002,Lupus,SLE Patient,None,34,SLE Patient
0,BFI-0010065,M456-S006,Lupus,SLE Patient,None,46,SLE Patient
0,BFI-0010067,M456-S008,Lupus,SLE Patient,None,49,SLE Patient


In [20]:
# sanity check: one entry per participant + specimen
assert all(
    metadata_df_c.groupby(["participant_label", "specimen_label"], observed=True).size()
    == 1
)

In [21]:
metadata_df_c.to_csv(
    config.paths.dataset_specific_metadata / "participant_specimen_disease_map.tsv",
    sep="\t",
    index=None,
)

In [22]:
metadata_df_c = pd.read_csv(
    config.paths.dataset_specific_metadata / "participant_specimen_disease_map.tsv",
    sep="\t",
)

In [23]:
metadata_df_c["disease_subtype"].value_counts()

HIV Non Neutralizing                               105
HIV Broad Neutralizing                              92
Healthy/Background - HIV Negative                   86
Healthy/Background - CMV Negative                   61
Healthy/Background - CMV Positive                   52
Covid19 - Admit                                     52
Healthy/Background (children)                       46
SLE Patient                                         43
Healthy/Background - SLE Negative                   31
Pediatric SLE - nephritis                           23
Pediatric SLE - no nephritis                        20
Covid19 - ICU                                       18
Covid19 - Sero-positive (ICU)                       18
SLE Multiple aAbs / SLE dsDNA WITH Nephritis        10
SLE Multiple aAbs                                    9
SLE Multiple aAbs / SLE dsDNA WITHOUT Nephritis      9
Covid19 - Sero-positive (Admit)                      6
Unaffected Control                                   6
Covid19 - 

Confirm HIV patient numbers -- we expect:

```
43	HIV Negative
46	HIV Broad Neutralizing
50	HIV Non Neutralizing
```

In [24]:
# Specimens
metadata_df_c[metadata_df_c["disease"] == "HIV"]["disease_subtype"].astype(
    "category"
).cat.remove_unused_categories().value_counts()

HIV Non Neutralizing      105
HIV Broad Neutralizing     92
Name: disease_subtype, dtype: int64

In [25]:
# Patients
metadata_df_c[metadata_df_c["disease"] == "HIV"].groupby(
    "disease_subtype", observed=True
)["participant_label"].nunique()

disease_subtype
HIV Broad Neutralizing    46
HIV Non Neutralizing      50
Name: participant_label, dtype: int64

In [26]:
# healthy specimens
metadata_df_c[metadata_df_c["disease"] == "Healthy/Background"][
    "disease_subtype"
].astype("category").cat.remove_unused_categories().value_counts()

Healthy/Background - HIV Negative    86
Healthy/Background - CMV Negative    61
Healthy/Background - CMV Positive    52
Healthy/Background (children)        46
Healthy/Background - SLE Negative    31
Unaffected Control                    6
Healthy/Background - CMV Unknown      1
Name: disease_subtype, dtype: int64

In [27]:
# healthy patients
metadata_df_c[metadata_df_c["disease"] == "Healthy/Background"].groupby(
    "disease_subtype", observed=True
)["participant_label"].nunique()

disease_subtype
Healthy/Background (children)        46
Healthy/Background - CMV Negative    61
Healthy/Background - CMV Positive    52
Healthy/Background - CMV Unknown      1
Healthy/Background - HIV Negative    43
Healthy/Background - SLE Negative    24
Unaffected Control                    6
Name: participant_label, dtype: int64

In [28]:
metadata_df_c[metadata_df_c["disease_subtype"] == "Healthy/Background - CMV Unknown"]

,participant_label,specimen_label,disease,disease_subtype,specimen_time_point,participant_age,participant_description
425,BFI-0003144,M64-095,Healthy/Background,Healthy/Background - CMV Unknown,NaN,22.0,Healthy Human Control


In [29]:
# covid specimens
metadata_df_c[metadata_df_c["disease"] == "Covid19"]["disease_subtype"].astype(
    "category"
).cat.remove_unused_categories().value_counts()

Covid19 - Admit                    52
Covid19 - ICU                      18
Covid19 - Sero-positive (ICU)      18
Covid19 - Sero-positive (Admit)     6
Covid19 - Acute 1                   5
Covid19 - Acute 2                   5
Covid19 - Convalescence             5
Covid19 - Sero-negative (Admit)     3
Covid19 - Sero-negative (ICU)       2
Name: disease_subtype, dtype: int64

In [30]:
# covid patients
metadata_df_c[metadata_df_c["disease"] == "Covid19"].groupby(
    "disease_subtype", observed=True
)["participant_label"].nunique()

disease_subtype
Covid19 - Acute 1                   5
Covid19 - Acute 2                   5
Covid19 - Admit                    41
Covid19 - Convalescence             5
Covid19 - ICU                      15
Covid19 - Sero-negative (Admit)     1
Covid19 - Sero-negative (ICU)       1
Covid19 - Sero-positive (Admit)     3
Covid19 - Sero-positive (ICU)       7
Name: participant_label, dtype: int64

In [31]:
client.shutdown()